# Wrappers: Bridge to External Libraries

This tutorial covers the `Wrapper` composition pattern in MolPy and demonstrates the `RDKitAdapter` with runnable examples.

**Learning objectives**
- Understand the purpose and design of `Wrapper` (composition-based integration).
- Use `RDKitAdapter` to bridge MolPy `Atomistic` objects and RDKit `Mol` objects.
- Perform bidirectional coordinate synchronization and use RDKit cheminformatics tools alongside MolPy operations.

## 1. Wrapper pattern (concept)

A `Wrapper` in MolPy is a composition-based adapter: it *holds* a MolPy `Atomistic` (or similar) and an external library object (e.g., an RDKit `Mol`), providing bidirectional synchronization and unified access.

Key expectations:
- `Wrapper` stores an internal MolPy object (accessible as `wrapper.internal`).
- It also stores an external object (e.g., `wrapper.mol` for RDKit).
- Provide `sync_to_external()` and `sync_to_internal()` methods for bidirectional data transfer.

## 2. RDKitAdapter: Basic Usage

The `RDKitAdapter` provides bidirectional synchronization between MolPy's `Atomistic` and RDKit's `Mol` objects.

**Prerequisites**
- Install the MolPy extra that enables the adapter: `molcrafts-molpy[rdkit_adapter]`
- Install RDKit (conda recommended): `conda install -c conda-forge rdkit`

In [ ]:
import molpy as mp
from molpy.core.atomistic import Atomistic
from molpy.external.rdkit_adapter import RDKitAdapter

try:
    from rdkit import Chem
    from rdkit.Chem import AllChem
except Exception as e:
    raise RuntimeError(
        "RDKit is required for this tutorial. Install it via conda-forge: "
        "conda install -c conda-forge rdkit"
    ) from e

# Example 1: Atomistic → RDKit
print("=== Example 1: Atomistic → RDKit ===")

# Create Atomistic structure (a tiny ethanol-like fragment)
atomistic = Atomistic()
c1 = atomistic.def_atom(symbol='C', x=0.0, y=0.0, z=0.0)
c2 = atomistic.def_atom(symbol='C', x=1.5, y=0.0, z=0.0)
o1 = atomistic.def_atom(symbol='O', x=2.25, y=1.0, z=0.0)
atomistic.def_bond(c1, c2, order=1.0)
atomistic.def_bond(c2, o1, order=1.0)

print(f"Initial Atomistic: {len(list(atomistic.atoms))} atoms")

# Create adapter and sync to RDKit
adapter = RDKitAdapter(internal=atomistic)
adapter.sync_to_external()

print(f"RDKit mol: {adapter.mol.GetNumAtoms()} atoms, {adapter.mol.GetNumBonds()} bonds")
print(f"SMILES: {Chem.MolToSmiles(adapter.mol)}")

# Example 2: Bidirectional sync
print("\n=== Example 2: Bidirectional sync ===")

# Modify coordinates in Atomistic and push to RDKit
for atom in atomistic.atoms:
    atom['x'] = atom.get('x', 0.0) + 1.0
adapter.sync_to_external()

conf = adapter.mol.GetConformer()
p0 = conf.GetAtomPosition(0)
print(f"First atom position after sync: ({p0.x:.2f}, {p0.y:.2f}, {p0.z:.2f})")

# Modify coordinates in RDKit and pull back into Atomistic
for idx in range(adapter.mol.GetNumAtoms()):
    pos = conf.GetAtomPosition(idx)
    conf.SetAtomPosition(idx, (pos.x, pos.y, pos.z + 0.5))
adapter.sync_to_internal()

first_atom = list(atomistic.atoms)[0]
print(f"First atom z-coordinate after sync back: {first_atom.get('z'):.2f}")

print("\n=== Bidirectional sync demonstrated! ===")

=== Example 1: Atomistic → RDKit ===
Initial Atomistic: 3 atoms
RDKit mol: 3 atoms, 2 bonds
SMILES: CCO

=== Example 2: Bidirectional Sync ===
First atom position after sync: (1.00, 0.00, 0.00)
First atom z-coordinate after sync back: 0.50

=== Bidirectional sync demonstrated! ===


## 3. Summary

- `RDKitAdapter` provides bidirectional synchronization between MolPy and RDKit.
- Use `sync_to_external()` to update the RDKit `Mol` from Atomistic.
- Use `sync_to_internal()` to update the Atomistic from RDKit `Mol`.
- This enables seamless integration of RDKit's cheminformatics tools with MolPy's molecular modeling capabilities.